In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root

In [4]:
import pandas as pd
import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    )
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
import transformers


# Loading the dataset

Since the data was already processed into a satisfying format, we simply need to import it via the dataset library function `load_dataset`.

In [5]:
dataset = load_dataset(
    "csv",
    data_files = '/content/drive/MyDrive/data/generator_training_data.csv',
    split = "train",
    #streaming = True
    )

In [6]:
dataset

Dataset({
    features: ['hotel_url', 'date', 'rating', 'title', 'text'],
    num_rows: 2500000
})

# Fine-tuning Llama 2

Norouzi (2023) describes an efficient way to handle Llama 2 despite the system RAM limitations that Google Colab comes with and was, therefore, used as the main source throughout this section. For more details see: [Mastering Llama 2: A Comprehensive Guide to Fine-Tuning in Google Colab](https://artificialcorner.com/mastering-llama-2-a-comprehensive-guide-to-fine-tuning-in-google-colab-bedfcc692b7f)

In [7]:
# Declaring the model name; as described in the paper, the 7 billion parameter version of Llama 2 is used
model_name = "meta-llama/Llama-2-7b-hf"

The parameters for the quantization config need to be defined:

In [8]:
# Load the entire model on the GPU 0
device_map = {"": 0}

# Set base model loading in 4-bits
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4); nf4 is shown to be better in the QLoRA paper
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [9]:
# Fetch the appropriate datatype from the torch library
compute_dtype= getattr(torch, bnb_4bit_compute_dtype)

# Initialize the quantization config using the previously declared parameters
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant
)

Now that we got both the `device_map` set to one GPU and the quantization config `bnb_config` initialized, the base model can be loaded into the system without any memory issues:

In [10]:
# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    quantization_config=bnb_config,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# Load the corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

As the original tokenizer has `pad_id` set to -1, we need to to define a custom padding token. While the [Llama 2 huggingface documentation](https://huggingface.co/docs/transformers/main/model_doc/llama2#overview) recommends using `tokenizer.add_special_tokens({"pad_token":"<pad>"})`, Norouzi points out that this can introduce CUDA-related errors. Therefore, he sees directly setting the `pad_token` as a safer option.




In [12]:
# Define a custom padding token
tokenizer.pad_token = "<PAD>"

# Set the padding direction to the right
tokenizer.padding_side = "right"

These paramaters are set in accordance to the QLoRA metod.

In [13]:
# LoRA attention dimension
lora_r = 64
# Alpha for LoRA scaling
lora_alpha = 64
# Dropout probability for LoRA
lora_dropout = 0.1

In [14]:
# Create the LoRA configuration
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    inference_mode=False,
    bias="none",
    task_type="CAUSAL_LM",
)

In [15]:
output_dir = "/content/drive/MyDrive/results"
final_checkpoint_dir = os.path.join(output_dir, "final_checkpoint")

Some of the training parameters; TODO: revisit this and optimize

In [16]:
num_train_epochs = 1
max_steps = -1
bf16 = False
fp16 = True
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
max_grad_norm = 0.3
optim = "adamw_bnb_8bit" # changed
learning_rate = 5e-5 # changed from 4e-5
lr_scheduler_type = "constant"
warmup_ratio = 0.03
weight_decay = 0.001
group_by_length = True
gradient_checkpointing = True
save_steps = 50
logging_steps = 10

In [17]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    #group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
)

In [18]:
max_seq_length = None
packing = True # for faster learning

In [19]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [20]:
resume_checkpoint = None

In [21]:
transformers.logging.set_verbosity_info()

In [23]:
trainer.train(resume_checkpoint)

***** Running training *****
  Num examples = 2,500,000
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 625,000
  Number of trainable parameters = 33,554,432
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.510100
20,2.446900


KeyboardInterrupt: ignored

In [ ]:
trainer.save_model(final_checkpoint_dir)